In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain.output_parsers import OutputFixingParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from prompts import load_template
from utils import DataVersionManager

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


.env 파일에 다음과 같은 형식으로 platform.openai.com에서 발급받은 토큰을 넣어주세요

```bash
HF_TOKEN={your_hf_token}    # 기존 .env
OPENAI_APIT_KEY={your_openai_token} # 추가할 토큰
```


In [ ]:
load_dotenv()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
REASONING = "reasoning"
CATEGORY = "category"
ANALYSIS = "analysis"

dvm = DataVersionManager()

In [ ]:
# 현재 실험은 0.1.x 입니다.
df = dvm.search_latest_experiments_data(1, 1, False)
df.head()

,id,paragraph,question,choices,answer,question_plus,reasoning
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)..."
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...


프롬프트는 역할 부여, Instruction prompting, 3-shots 기법을 사용했습니다.


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="information_source_with_reasoning.txt", template_type="data_classification")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

output_parser = JsonOutputParser()

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

domain_classifier = prompt | llm | fixing_parser

In [ ]:
def classify_problem(data: pd.Series) -> tuple[str, str]:
    """프롬프트에 따라 LLM이 문제 데이터를 분류하고 결과를 문자열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        tuple(str, str): '내적 추론' 혹은 '외적 추론'으로 분류된 카테고리 명
    """
    response = domain_classifier.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
            "reasoning": data[REASONING],
        },
    )
    return response[CATEGORY], response[ANALYSIS]


tqdm.pandas()
df["result"] = df.progress_apply(classify_problem, axis=1)

# 결과 분리
df[CATEGORY] = df["result"].apply(lambda x: x[0])
df[ANALYSIS] = df["result"].apply(lambda x: x[1])

# 임시 열 삭제
df.drop(columns=["result"], inplace=True)

df.head()

100%|██████████| 1627/1627 [40:04<00:00,  1.48s/it] 


,id,paragraph,question,choices,answer,question_plus,reasoning,category,analysis
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,외적 추론,Reasoning 과정에서 조선 후기 붕당의 역사적 맥락과 예송논쟁에 대한 배경 지...
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",외적 추론,Reasoning 과정에서 고려사와 관련된 역사적 사실 및 맥락이 필요합니다. 특히...
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,외적 추론,Reasoning 과정에서 소정방과 김춘추의 관계 및 역사적 사건에 대한 배경 지식...
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,외적 추론,Reasoning 과정에서 (가) 인물에 대한 역사적 배경과 영조의 정책에 대한 외...
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,외적 추론,"Reasoning 과정에서 발해와 고구려의 역사적 맥락, 그리고 중앙 행정 기구인 ..."


간단한 실험을 위해 20개 데이터를 추출하여 결과를 보겠습니다.


In [ ]:
# 현재 실험은 0.1.x 입니다.
df.to_csv(dvm.update_file_path(0, 1, save_in_experiment=True), index=False)

`data/experiments/train_{실험_버전}.csv` 파일로 저장된 위 실험 결과를 다음 과정에 따라 확인해주시길 바랍니다.

1. 프로젝트 디렉토리에서 터미널을 열어 아래 명령어를 실행

```bash
$ streamlit run analysis_dashboard.py
```

2. 실험 데이터 페이지에서 데이터 분석
